In [1]:
from proj_utils.fake_input_generator import generator
from proj_utils.input_interpreter import format_input

import matplotlib.pyplot as plt

from VRP.greedy_solutions import nearest_neighbour_solution
from VRP.modify_solution import combined_rand_modification
from VRP.verify_solution import valid_path, calculate_path_distance
from VRP.compare_solutions import num_different_edges


from proj_utils.path import print_path



import simulated_annealing.solver as SA
import taboo_search.solver as TS

from multi_agent.solver_model import MultiAgentSolverModel
from multi_agent.colaboration_types import ColaborationTypes
from solver_step import SolverStep
from multi_agent.solution_pool import solution_pool_simple
%matplotlib widget

In [2]:
truck_volume_capacity = 15
truck_weight_capacity = 10
truck_capacity = (truck_volume_capacity, truck_weight_capacity)
dists, demand, points = generator(40)

In [3]:
eval_function = lambda x: calculate_path_distance(x, dists) if(valid_path(x,truck_capacity,demand)) else float('inf')

In [4]:
solver_step_SA = SA.generic_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    )

In [5]:
taboo_list_size = 10
solver_step_TS = TS.taboo_search_solver_factory(
    lambda x: combined_rand_modification(x, len(points)),
    eval_function,
    taboo_list_size
    )

In [6]:
init_state = nearest_neighbour_solution(dists, demand, truck_capacity)
init_step = SolverStep(init_state,eval_function(init_state),100)
step_function_list = [solver_step_SA, solver_step_TS]

In [7]:
solution_pool = solution_pool_simple(num_different_edges, radious=2)
sol_model = MultiAgentSolverModel(lambda : init_step, step_function_list, 2946091,20000,30,solution_pool,ColaborationTypes.FRIENDS)

/opt/miniconda3/lib/python3.12/site-packages/mesa/time.py:82: FutureWarning: The AgentSet is experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1919
  self._agents: AgentSet = AgentSet(agents, model)


In [8]:
for _ in range(100):
    sol_model.step()

population generated


In [9]:
sol_states = sol_model.datacollector.get_model_vars_dataframe()
print(sol_states['TheGlobalBest'][0])

[0, 1, 29, 6, 28, 20, 14, 16, 38, 33, 13, 0, 3, 23, 25, 7, 26, 8, 17, 9, 24, 18, 0, 15, 37, 30, 12, 34, 32, 19, 10, 35, 31, 0, 11, 39, 21, 27, 5, 22, 36, 4, 2, 0]
